## Anaconda Env Setting

```
conda create -n some_env_name
conda install python=3.10 numpy pytorch torchvision pandas scikit-learn jupyter notebook
```

In [1]:
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
import numpy as np
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import randint
from sklearn.model_selection import GridSearchCV
import pandas as pd

## Download Resources

In [2]:
# MNIST
mnist_train_transform = transforms.Compose ([transforms.ToTensor()])
mnist_test_transform = transforms.Compose([transforms.ToTensor()])

trainset_mnist = datasets.MNIST(root = './data', train = True, download = True, transform = mnist_train_transform)
testset_mnist = datasets.MNIST(root = './data', train = False, download = True, transform = mnist_test_transform)

MNIST_train = DataLoader(trainset_mnist, batch_size=32, shuffle=True, num_workers = 2)
MNIST_test = DataLoader(testset_mnist, batch_size=32, shuffle=False, num_workers=2)

MNIST_train_images = []
MNIST_train_labels = []
for batch in MNIST_train:
  images, labels = batch
  images_flat = images.view(images.shape[0], -1)
  MNIST_train_images.append(images_flat.numpy())
  MNIST_train_labels.append(labels.numpy())
MNIST_train_images = np.vstack(MNIST_train_images)
MNIST_train_labels = np.concatenate(MNIST_train_labels)

MNIST_test_images = []
MNIST_test_labels = []
for batch in MNIST_test:
  images, labels = batch
  images_flat = images.view(images.shape[0],-1)
  MNIST_test_images.append(images_flat.numpy())
  MNIST_test_labels.append(labels.numpy())
MNIST_test_images = np.vstack(MNIST_test_images)
MNIST_test_labels = np.concatenate(MNIST_test_labels)

## Decision Tree Test

In [3]:
model = DecisionTreeClassifier(max_depth=12)
model.fit(MNIST_train_images, MNIST_train_labels)
score = model.score(MNIST_test_images, MNIST_test_labels)
print("Model %d score : %f\nParameters : %s" % (model.max_depth, score, model.get_params()))

Model 12 score : 0.878700
Parameters : {'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': 12, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'random_state': None, 'splitter': 'best'}


## Decision Tree Assignment

In [4]:
from sklearn.model_selection import cross_validate
params_grid = {
  'min_samples_split': [2, 5, 10],
  'min_samples_leaf': [1, 2, 4], 
  'max_leaf_nodes': [5, 10, None]
}

grid_searchs=[]
result=[]
for depth in [3,6,9,12]:
    model = DecisionTreeClassifier(max_depth=depth)
    grid_search = GridSearchCV(model, param_grid=params_grid, cv=5)
    grid_search.fit(MNIST_train_images, MNIST_train_labels)
    
    accuracy_training = grid_search.score(MNIST_train_images, MNIST_train_labels)
    accuracy_test = grid_search.score(MNIST_test_images, MNIST_test_labels)
    result.append([depth, accuracy_training, accuracy_test])
    grid_searchs.append(grid_search)
    print('depth %d best score : %f' % (depth, grid_search.best_score_))

pd.DataFrame(result, columns=["Depth", "Accuracy\n(Training Set)", "Accuracy\n(Test Set)"])

depth 3.000000 best score : 0.483100
depth 6.000000 best score : 0.737550
depth 9.000000 best score : 0.837583
depth 12.000000 best score : 0.867850


,Depth,Accuracy\n(Training Set),Accuracy\n(Test Set)
0,3,0.491517,0.4953
1,6,0.738250,0.7415
2,9,0.865483,0.8500
3,12,0.949167,0.8763


## SVM Test

## SVM Assignment